In [23]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.update()
Pkg.resolve()
Pkg.instantiate()
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo
using LinearAlgebra
import Plots
using Random
using JLD2
using Test
using StaticArrays
using Printf
using MuJoCo
using Distributions

  Activating environment at `c:\Users\AiPEX-WS4\Documents\ocrl-piano\Julia_fmincon_tests\Project.toml`
    Updating registry at `C:\Users\AiPEX-WS4\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Installed Ipopt ──────────────────────── v1.6.3
   Installed IJulia ─────────────────────── v1.24.2
   Installed BangBang ───────────────────── v0.4.2
   Installed FileIO ─────────────────────── v1.16.3
   Installed StaticArrays ───────────────── v1.9.5
   Installed Plots ──────────────────────── v1.40.4
   Installed Distributions ──────────────── v0.25.109
   Installed GLFW ───────────────────────── v3.4.1
   Installed ECOS ───────────────────────── v1.1.2
   Installed JLD2 ───────────────────────── v0.4.48
   Installed Missings ───────────────────── v1.2.0
   Installed PrettyTables ───────────────── v2.3.2
   Installed RelocatableFolders ─────────── v1.0.1
   Installed libinput_jll ───────────────── v1.18.0+0
   Installed DensityInterface 

In [2]:
# include fmincon for ipopt
include(joinpath(@__DIR__, "utils","fmincon.jl"))

LoadError: LoadError: ArgumentError: Package MathOptInterface [b8f27783-ece8-5eb3-8dc8-9495eed66fee] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.

in expression starting at c:\Users\AiPEX-WS4\Documents\ocrl-piano\Julia_fmincon_tests\utils\fmincon.jl:1

In [3]:
# import piano model
model = load_model("piano_with_one_shadow_hand_small/scene.xml")
# model = load_model("piano_with_one_shadow_hand_simplified/scene.xml")
data = init_data(model)

UndefVarError: UndefVarError: load_model not defined

Model info

In [4]:
println("number of geoms: ",model.ngeom)
println("number of controls: ",model.nu)
println("number of states: ",model.nv+model.nq)
forward!(model, data)
println("Simulation timestep: ", model.opt.timestep)
println("Positions of joints: ", data.qpos)
println("Cartesion position of joints: ", data.geom_xpos)
println("Size of Cartesion position of joints: ", size(data.geom_xpos))
println("model ")
finger_geom_indices = [42,50,58,68,77]
"""
index finger:
    name:
        rh_shadow_hand//unnamed_geom_25
        rh_shadow_hand//unnamed_geom_26
    index:
        42
        43
    range:
        x:
        y:-0.6-0.5
        z:0-0.3

middle finger:
    name:
        rh_shadow_hand//unnamed_geom_33
        rh_shadow_hand//unnamed_geom_34
    index:
        50
        51
ring finger:
    name:
        rh_shadow_hand//unnamed_geom_41
        rh_shadow_hand//unnamed_geom_42
    index:
        58
        59
pinky finger:
    name:
        rh_shadow_hand//unnamed_geom_51
        rh_shadow_hand//unnamed_geom_52
    index:
        68
        69
thumb:
    name:
        rh_shadow_hand//unnamed_geom_60
        rh_shadow_hand//unnamed_geom_61
    index:
        77
        78
""";

UndefVarError: UndefVarError: model not defined

Visualize target finger position

In [5]:
# control indices:
# 1: wrist left right
# 2: wrist up down
# 8, 9, 10: index finger joints
# 21: whole hand left right 
# 22: whole hand front back

In [6]:
reset!(model, data)
nx = model.nq + model.nv + model.na # State vector dimension

states = zeros(nx, 11)
states_geom = zeros(3, 11)
U = zeros(model.nu)
U[4] = 0.664
U[5] = 1.1
U[2] = 0.21
# U[1] = -0.234
record_t_3 = zeros(nx)
data = init_data(model)
step!(model, data)
println(get_physics_state(model, data))
data = init_data(model)
step!(model, data)
println(get_physics_state(model, data))

step!(model, data)
# println("max",maximum(get_physics_state(model, data)))
# println("min",minimum(get_physics_state(model, data)))

states[:,1] = get_physics_state(model, data)
states_geom[:,1] = data.geom_xpos[finger_geom_indices[1],:]
for t in 1:10
    # println(t,data.geom_xpos[finger_geom_indices[1],:])
    data.ctrl .= U
    step!(model, data)
    
    states[:,t+1] = get_physics_state(model, data)
    states_geom[:,t+1] = data.geom_xpos[finger_geom_indices[1],:]
end
# println(data.geom_xpos[finger_geom_indices[1],:])
reset!(model, data)

# data.ctrl .= U
data.qpos .= states[:,2][1:model.nq]
data.qvel .= states[:,2][model.nq+1:end]
forward!(model, data)
# println("hi")
println("hi",data.geom_xpos[finger_geom_indices[1],:])
for t in 1:11
    println(t,states_geom[:,t])
end
# init_visualiser()
# visualise!(model, data, trajectories = [states])

UndefVarError: UndefVarError: reset! not defined

## Test DIRCOL: moving index finger left right

In [7]:
global mj_access_count = 0

0

In [8]:
function normalize_states(model, x)
    # get the states and normalize according to joint and control ranges
    for j in 
        x_l[idx.x[i][j]] = model.jnt_range[j,1]
        x_u[idx.x[i][j]] = model.jnt_range[j,2]
end

ErrorException: syntax: unexpected "="

In [9]:
function wrapped_mj_step(model, data, x, uk, forward)
    # given the current model and data. set the state and control to the model and perform a forward step
    # data.ctrl .= zeros(model.nu)
    global mj_access_count += 1
    if typeof(x) == Vector{Float64}
        # set control 
        data.ctrl[:] .= uk
        # set state
        # data.qpos .= x[1:model.nq]
        # data.qvel .= x[(model.nq + 1):end]
        data.qpos .= x[1:model.nq]
        data.qvel .= x[model.nq+1:end]
    else
        # if using diff types, we need to convert the dual numbers to floats
        converted_x = ForwardDiff.value.(x)
        # set control
        data.ctrl[:] .= ForwardDiff.value.(uk)

        # set state
        data.qpos .= converted_x[1:model.nq]
        data.qvel .= converted_x[(model.nq + 1):end]
    end
    
    # take discrete dynamics step 
    if forward
        forward!(model,data)
    else
        step!(model, data) 
    end

    # return updated state k + 1
    # xkp1 = zeros(model.nq + model.nv) 
    # xkp1 .= get_physics_state(model, data)
    # finger_coordinates = data.geom_xpos[finger_geom_indices,:]
    
    return data.geom_xpos[finger_geom_indices[1],:], get_physics_state(model, data)
end

function robohand_cost(params::NamedTuple, Z::Vector)::Real
    # TODO: implement cost function
    idx, N, xg, xic = params.idx, params.N, params.xg, params.xic
    model = params.model
    data = init_data(model)
    step!(model, data)
    Q, R, R_d, Q_reg, Q_coord = params.Q, params.R, params.R_d, params.Q_reg, params.Q_coord
    coord_interpolation = params.coord_interpolation

    # stage cost
    cost = 0.0
    xi = Z[idx.x[1]]
    ui = Z[idx.u[1]]

    finger_geom_xpos, _ = wrapped_mj_step(model, data, xi, ui, true)
    cost += 0.5*(finger_geom_xpos-xg)'*Q*(finger_geom_xpos-xg)+0.5*ui'*R*ui
    cost += 0.5*(xi-xic)'*Q_reg*(xi-xic)
    cost += 0.5*(finger_geom_xpos-coord_interpolation[1])'*Q_coord*(finger_geom_xpos-coord_interpolation[1])
    # cost += 0.5*(xi)'*Q_reg*(xi)
    for i = 2:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        ui_1 = Z[idx.u[i-1]]
        finger_geom_xpos, _ = wrapped_mj_step(model, data, xi, ui, true)
        # @show xi-xg
        # actual cost
        cost += 0.5*(finger_geom_xpos-xg[i])'*Q*(finger_geom_xpos-xg)+0.5*ui'*R*ui#+0.5*(ui-ui_1)'*R_d*(ui-ui_1)
        # regularizer
        cost += 0.5*(xi-xic)'*Q_reg*(xi-xic)
        cost += 0.5*(finger_geom_xpos-coord_interpolation[i])'*Q_coord*(finger_geom_xpos-coord_interpolation[i])
    end
    cost += 0.5*(finger_geom_xpos-coord_interpolation[N])'*Q_coord*(finger_geom_xpos-coord_interpolation[N])
    # # terminal cost 
    # xf = Z[idx.x[N]]
    # # cost += 0.5*(xf[94:96]-xg[94:96])'*Qf*(xf[94:96]-xg[94:96])
    # finger_geom_xpos, _ = wrapped_mj_step(model, data, xf, 0, false)
    # cost += 0.5*(finger_geom_xpos-xg)'*Qf*(finger_geom_xpos-xg)

    return cost
end
function robohand_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    model = params.model
    data = init_data(model)
    step!(model, data)
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        _,  xip1_mujoco = wrapped_mj_step(model, data, xi, ui, false)

        c[idx.c[i]] = (xip1_mujoco - xip1)
    end
    # println(typeof(c))
    return c 
end

function robohand_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement equality constraints
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    model = params.model
    
    # initial_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[1]], Z[idx.u[1]], true)
    terminal_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[N-1]], Z[idx.u[N-1]] , true)
    

    con_1 =  Z[idx.x[1]] - xic
    
    con_2 =  terminal_pose_index - xg
    # con_2[3] = con_2[3] - 0.5

    return [con_1; con_2; robohand_dynamics_constraints(params, Z)]
    # return zeros(eltype(Z), 0)
end
function robohand_inequality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement inequality constraints
    # println(Z)
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    model = params.model
    
    # # initial_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[1]], Z[idx.u[1]], true)
    # terminal_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[N-1]], Z[idx.u[N-1]] , true)
    

    # con_1 =  Z[idx.x[1]] - xic
    
    # cart_z_end =  terminal_pose_index 

    # return [con_1; con_2; robohand_dynamics_constraints(params, Z)]
    return zeros(eltype(Z), 0)
    # return cart_z_end
end
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

create_idx (generic function with 1 method)

## test the objective functions outputs

In [10]:
function robohand_cost_test(params::NamedTuple, Z::Vector)
    # TODO: implement cost function
    idx, N, xg, xic = params.idx, params.N, params.xg, params.xic
    model = params.model
    data = init_data(model)
    step!(model, data)
    Q, R, R_d, Q_reg, Q_coord = params.Q, params.R, params.R_d, params.Q_reg, params.Q_coord
    coord_interpolation = params.coord_interpolation

    # stage cost
    cost = 0.0
    xi = Z[idx.x[1]]
    ui = Z[idx.u[1]]
    # stage cost Vector
    stage_vec = []

    finger_geom_xpos, _ = wrapped_mj_step(model, data, xi, ui, true)
    cost += 0.5*(finger_geom_xpos-xg)'*Q*(finger_geom_xpos-xg)+0.5*ui'*R*ui
    # cost += 0.5*(xi-xic)'*Q_reg*(xi-xic)
    cost += 0.5*(finger_geom_xpos-coord_interpolation[1])'*Q_coord*(finger_geom_xpos-coord_interpolation[1])
    push!(stage_vec, cost)
    cost += 0.5*(xi)'*Q_reg*(xi)
    for i = 2:(N-1)
        cost = 0.0
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        ui_1 = Z[idx.u[i-1]]
        finger_geom_xpos, _ = wrapped_mj_step(model, data, xi, ui, true)
        # @show xi-xg
        # actual cost
        cost += 0.5*(finger_geom_xpos-xg)'*Q*(finger_geom_xpos-xg)+0.5*ui'*R*ui#+0.5*(ui-ui_1)'*R_d*(ui-ui_1)
        # regularizer
        cost += 0.5*(xi-xic)'*Q_reg*(xi-xic)
        cost += 0.5*(finger_geom_xpos-coord_interpolation[i])'*Q_coord*(finger_geom_xpos-coord_interpolation[i])
        push!(stage_vec, cost)
    end
    cost = 0.0
    cost += 0.5*(finger_geom_xpos-xg)'*Q*(finger_geom_xpos-xg)+0.5*ui'*R*ui
    cost += 0.5*(xi-xic)'*Q_reg*(xi-xic)
    cost += 0.5*(finger_geom_xpos-coord_interpolation[N])'*Q_coord*(finger_geom_xpos-coord_interpolation[N])
    push!(stage_vec, cost)
    # # terminal cost 
    # xf = Z[idx.x[N]]
    # # cost += 0.5*(xf[94:96]-xg[94:96])'*Qf*(xf[94:96]-xg[94:96])
    # finger_geom_xpos, _ = wrapped_mj_step(model, data, xf, 0, false)
    # cost += 0.5*(finger_geom_xpos-xg)'*Qf*(finger_geom_xpos-xg)

    return stage_vec
end
# instantiate model and data
model = load_model("piano_with_one_shadow_hand_small/scene.xml")
# model = load_model("piano_with_one_shadow_hand_simplified/scene.xml")

data = init_data(model)

# reset the model and data
reset!(model, data)

# initiate time and time steps
dt = 0.01
tf = 0.1
t_vec = 0:dt:tf
N = length(t_vec)

# LQR cost
# Q size of full states
# Q = diagm(ones(model.nq + model.nv))
# R = 0.1*diagm(ones(model.nu))
# Qf = 10*diagm(ones(model.nq + model.nv))

# Q = 100*diagm(ones(3))
Q = 1*diagm([5,5,30])
Q_coord = 1*diagm(ones(3))
Q_reg = 0.0001*diagm(ones(model.nq + model.nv))
R = 1*diagm(ones(model.nu))
# udifference regularizer
R_d = 10*diagm(ones(model.nu))

idx = create_idx(model.nq + model.nv, model.nu, N)
# initial and goal states
step!(model, data)
z0 = randn(idx.nz)
xic = get_physics_state(model, data)
z0 = randn(idx.nz)
z0[idx.x[1]] .= xic

xic_geom = data.geom_xpos[finger_geom_indices[1],:]
U_ref = zeros(model.nu)
U_ref[4] = 0.664
U_ref[5] = 1.1
U_ref[2] = 0.21
for t in 1:N-1
    data.ctrl .= U_ref
    step!(model, data)
    z0[idx.x[t+1]] .= get_physics_state(model, data)
    z0[idx.u[t]] .= U_ref
end
# initial and goal state
xg_physics = get_physics_state(model, data)
xg = data.geom_xpos[finger_geom_indices[1],:]
# reset the model and data


# linear-wise interpolation of xi and xg
coord_interpolation = [zeros(3) for _ in 1:N]
for i in 1:N
    coord_interpolation[i] = xic_geom + (i-1)/(N-1)*(xg-xic_geom)
end

reset!(model, data)

params = (Q = Q, R = R, R_d = R_d, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, Q_reg=Q_reg, coord_interpolation=coord_interpolation, Q_coord=Q_coord)

a = robohand_cost_test(params, z0)
println(a)
# println(N)
# display(plot(t_vec, a, label = ["θ1" "θ2" "θ3" ], xlabel = "time (s)", title = "State Trajectory"))

UndefVarError: UndefVarError: load_model not defined

In [11]:
global mj_access_count = 0
function solve_finger_moving(;verbose=true)
    # instantiate model and data
    model = load_model("piano_with_one_shadow_hand_small/scene.xml")
    # model = load_model("piano_with_one_shadow_hand_simplified/scene.xml")

    data = init_data(model)

    # reset the model and data
    reset!(model, data)

    # initiate time and time steps
    dt = 0.01
    tf = 0.1
    t_vec = 0:dt:tf
    N = length(t_vec)

    # LQR cost
    # Q size of full states
    # Q = diagm(ones(model.nq + model.nv))
    # R = 0.1*diagm(ones(model.nu))
    # Qf = 10*diagm(ones(model.nq + model.nv))

    # Q = 100*diagm(ones(3))
    Q = 1*diagm([5,5,30])
    Q_coord = 1*diagm(ones(3))
    Q_reg = 1*diagm(ones(model.nq + model.nv))
    R = 1*diagm(ones(model.nu))
    # udifference regularizer
    R_d = 10*diagm(ones(model.nu))

    idx = create_idx(model.nq + model.nv, model.nu, N)
    # initial and goal states
    step!(model, data)
    xic = get_physics_state(model, data)
    xic_geom = data.geom_xpos[finger_geom_indices[1],:]
    U_ref = zeros(model.nu)
    U_ref[4] = 0.664
    U_ref[5] = 1.1
    U_ref[2] = 0.21
    for t in 1:N-1
        data.ctrl .= U_ref
        step!(model, data)
    end
    # initial and goal state
    xg_physics = get_physics_state(model, data)
    xg = data.geom_xpos[finger_geom_indices[1],:]
    # reset the model and data

    
    # linear-wise interpolation of xi and xg
    coord_interpolation = [zeros(3) for _ in 1:N]
    for i in 1:N
        coord_interpolation[i] = xic_geom + (i-1)/(N-1)*(xg-xic_geom)
    end

    reset!(model, data)
    params = (Q = Q, R = R, R_d = R_d, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, Q_reg=Q_reg, coord_interpolation=coord_interpolation, Q_coord=Q_coord)

    # primal bounds
    x_l = -1 * Inf * ones(idx.nz)
    x_u = Inf*ones(idx.nz)
    for i = 1:N
        for j = 1:model.nq
            # hands
            x_l[idx.x[i][j]] = model.jnt_range[j,1]
            x_u[idx.x[i][j]] = model.jnt_range[j,2]
        end
    end
    for i = 1:(N-1)
        for j = 1:model.nu
            x_l[idx.u[i][j]] = model.actuator_ctrlrange[j,1]
            x_u[idx.u[i][j]] = model.actuator_ctrlrange[j,2]
        end
        # for j in [2,5,4]
        #     x_l[idx.u[i][j]] = 0
        #     x_u[idx.u[i][j]] = model.actuator_ctrlrange[j,2]
        # end
    end
    

    # inequality constraints
    # c_l = xg .- 0.005
    # c_u = xg .+ 0.005
    c_l = zeros(0)
    c_u = zeros(0)
    # c_l = -0.01*ones(idx.nx+3+idx.nx*(N-1))
    # c_u = 0.01*ones(idx.nx+3+idx.nx*(N-1))
    # initial guess 
    z0 = randn(idx.nz)
    z0[idx.x[1]] .= xic
    for i = 2:N
        # z0[idx.x[i]] .= xic + ((xg_physics-xic)/(N))*(i)
        z0[idx.x[i]] .= xic
        # z0[idx.x[i]] .= xg_physics
        # for j = 1:model.nq
        #     z0[idx.x[i][j]] = rand(Uniform(model.jnt_range[j,1],model.jnt_range[j,2]))
        # end
    end
    println(length(robohand_inequality_constraints(params,z0)))
    println(length(c_u))
    for i = 1:N-1
        for j = 1:model.nu
            z0[idx.u[i][j]] = rand(Uniform(model.actuator_ctrlrange[j,1],model.actuator_ctrlrange[j,2]))
        end
        # for j in [2,5,4]
        #     z0[idx.u[i][j]] = rand(Uniform(model.actuator_ctrlrange[j,1],model.actuator_ctrlrange[j,2]))
        # end
    end
    # diff type
    # diff_type = :auto 
    diff_type = :finite
    
    
    Z = fmincon(robohand_cost,robohand_equality_constraints,robohand_inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-4, c_tol = 1e-4, max_iters = 500, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    return X, U, t_vec, params
end

solve_finger_moving (generic function with 1 method)

In [12]:
X, U, t_vec, params= solve_finger_moving(verbose=true)

UndefVarError: UndefVarError: load_model not defined

In [13]:
mj_access_count 

0

In [14]:
X

UndefVarError: UndefVarError: X not defined

In [15]:
U
# U2[4] = 0.664
# U2[5] = 1.1
# U2[2] = 0.21

UndefVarError: UndefVarError: U not defined

In [16]:
# instantiate model and data
# model = load_model("piano_with_one_shadow_hand_simplified/scene.xml")
model = load_model("piano_with_one_shadow_hand_small/scene.xml")
data = init_data(model)

# reset the model and data j in [2,5,4]
        #     z0[idx.u[i][j]] = rand(Uniform(model.actuator_ctrlrange[j,1],model.actuator_ctrlrange[j,2]))
        # end
dt = 0.01
tf = 0.1
t_vec = 0:dt:tf
N = length(t_vec)
states = zeros(nx, N)
ctrl_states = zeros(model.nu, N)
reset!(model, data)
step!(model, data)
states[:,1] = get_physics_state(model, data)
for t in 1:N-1
    data.ctrl .= U[t][:]
    step!(model, data)
    states[:,t+1] = get_physics_state(model, data)
end
println("end using controls:",data.geom_xpos[finger_geom_indices[1],:])
states3 = zeros(nx, N)
for t in 1:N
    states3[:,t] = X[t][:]
end

reset!(model, data)
data.ctrl .=0
data.qpos .= states3[1:model.nq,N]
data.qvel .= states3[model.nq+1:end,N]
forward!(model, data)
final_cart_fmincon = data.geom_xpos[finger_geom_indices[1],:]
reset!(model, data)
nx = model.nq + model.nv + model.na # State vector dimension

states2 = zeros(nx, N)
states2[:,1] = get_physics_state(model, data)
states2_coord = zeros(3,N)
states2_coord[:,1] = data.geom_xpos[finger_geom_indices[1],:]
U2 = zeros(model.nu)
U2[4] = 0.664
U2[5] = 1.1
U2[2] = 0.21
for t in 1:N-1
    data.ctrl .= U2
    step!(model, data)
    states2[:,t+1] = get_physics_state(model, data)
    states2_coord[:,t+1] = data.geom_xpos[finger_geom_indices[1],:]
end
println(data.geom_xpos[finger_geom_indices[1],:])
println(norm((final_cart_fmincon[3]-data.geom_xpos[finger_geom_indices[1],3]),2))
for t in 1:N
    println(states2_coord[:,t])
end

UndefVarError: UndefVarError: load_model not defined

In [17]:
init_visualiser()
visualise!(model, data, trajectories = [states, states2])

UndefVarError: UndefVarError: init_visualiser not defined

In [18]:
init_visualiser()
visualise!(model, data, trajectories = [states2])

UndefVarError: UndefVarError: init_visualiser not defined

In [19]:
init_visualiser()
visualise!(model, data, trajectories = [states3])

UndefVarError: UndefVarError: init_visualiser not defined